In [1]:
import sqlite3
import csv
conn = sqlite3.connect("/home/pkosewski/python/pd4/databases/flights_full.sql")

In [1]:
#import sqlite3
#import csv
#conn = sqlite3.connect("/home/pkosewski/python/pd4/databases/processed.sql")

In [4]:
conn.execute("""
CREATE TABLE num_filghts_d AS 
    SELECT ontime.year            AS year,
            ontime.month          AS month,
            ontime.DayofMonth     AS day,
            origin.latitude_deg   AS origin_lat, 
            origin.longitude_deg  AS origin_lon, 
            origin.name           AS origin_name, 
            dest.latitude_deg     AS dest_lat, 
            dest.longitude_deg    AS dest_lon, 
            dest.name             AS dest_name, 
            Count(*)              AS number_of_flights 
    FROM ontime 
    LEFT JOIN airports AS origin ON ontime.Origin == origin.iata
    LEFT JOIN airports AS dest   ON ontime.Dest   == dest.iata
    GROUP BY ontime.year,
            ontime.month,
            ontime.DayofMonth,
            ontime.Origin,
            ontime.Dest;
""")
conn.commit()

In [7]:
conn.execute("""
CREATE TABLE num_filghts_m AS 
    SELECT ontime.year            AS year,
            ontime.month          AS month,
            origin.latitude_deg   AS origin_lat, 
            origin.longitude_deg  AS origin_lon, 
            origin.name           AS origin_name, 
            dest.latitude_deg     AS dest_lat, 
            dest.longitude_deg    AS dest_lon, 
            dest.name             AS dest_name, 
            Count(*)              AS number_of_flights 
    FROM ontime 
    LEFT JOIN airports AS origin ON ontime.Origin == origin.iata
    LEFT JOIN airports AS dest   ON ontime.Dest   == dest.iata
    GROUP BY ontime.year,
            ontime.month,
            ontime.Origin,
            ontime.Dest;
""")
conn.commit()

In [10]:
cur = conn.execute("""
SELECT * FROM num_filghts_d LIMIT 1
""")
cur.fetchall()

[(1987,
  10,
  1,
  40.652099609375,
  -75.44080352783203,
  'Lehigh Valley International Airport',
  33.63669967651367,
  -84.4281005859375,
  'Hartsfield Jackson Atlanta International Airport',
  2)]

In [11]:
cur = conn.execute("""
SELECT * FROM num_filghts_m LIMIT 1
""")
cur.fetchall()

[(1987,
  10,
  40.652099609375,
  -75.44080352783203,
  'Lehigh Valley International Airport',
  33.63669967651367,
  -84.4281005859375,
  'Hartsfield Jackson Atlanta International Airport',
  60)]

In [12]:
cur = conn.execute("""
SELECT count(*) FROM num_filghts_d
""")
cur.fetchall()

[(25352185,)]

In [13]:
cur = conn.execute("""
SELECT count(*) FROM num_filghts_m
""")
cur.fetchall()

[(861628,)]

In [14]:
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('ontime',), ('airports_all',), ('airports',), ('num_filghts_d',), ('num_filghts_m',)]


In [15]:
# tabele wynikowe
import pandas as pd
import sqlite3

conn_processed = sqlite3.connect("/home/pkosewski/python/pd4/databases/processed.sql")
conn           = sqlite3.connect("/home/pkosewski/python/pd4/databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from num_filghts_d", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("num_filghts_d", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()

MemoryError: 

In [16]:
# tabele wynikowe
import pandas as pd
import sqlite3

conn_processed = sqlite3.connect("/home/pkosewski/python/pd4/databases/processed.sql")
conn           = sqlite3.connect("/home/pkosewski/python/pd4/databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from num_filghts_m", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("num_filghts_m", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()

In [2]:
cur = conn.execute("""
SELECT COUNT(*) FROM airports
""")
cur.fetchall()

[(9103,)]

# Mean departure delay

In [4]:
conn.execute("""
CREATE TABLE US_filghts_dep_delay AS 
    SELECT ontime.year                              AS year,
            ontime.month                            AS month,
            origin.latitude_deg                     AS origin_lat, 
            origin.longitude_deg                    AS origin_lon, 
            origin.name                             AS origin_name, 
            ontime.Origin                           AS origin_id, 
            dest.latitude_deg                       AS dest_lat, 
            dest.longitude_deg                      AS dest_lon, 
            dest.name                               AS dest_name,
            ontime.Dest                             AS dest_id, 
            AVG(ontime.DepDelay)                    AS DepDelay, 
            AVG(ontime.ArrDelay - ontime.DepDelay)  AS delay_gain 
    FROM ontime 
    LEFT JOIN airports AS origin ON ontime.Origin == origin.iata
    LEFT JOIN airports AS dest   ON ontime.Dest   == dest.iata
    WHERE origin.country == 'US' AND dest.country == 'US' 
    GROUP BY ontime.year,
            ontime.month,
            ontime.Origin,
            ontime.Dest
""")
conn.commit()

In [5]:
cur = conn.execute("""
SELECT COUNT(*) FROM US_filghts_dep_delay
""")
cur.fetchall()

[(680007,)]

In [6]:
cur = conn.execute("""
SELECT COUNT(*) FROM US_filghts_dep_delay where DepDelay > 15
""")
cur.fetchall()

[(82297,)]

In [8]:
cur = conn.execute("""
Select count(*) From (
           SELECT origin_name FROM US_filghts_dep_delay group by origin_name
           )
""")
cur.fetchall()

[(133,)]

In [3]:
conn.execute("""
CREATE TABLE US_filghts_dep_delay_Y AS 
    SELECT ontime.year                              AS year,
            origin.latitude_deg                     AS origin_lat, 
            origin.longitude_deg                    AS origin_lon, 
            origin.name                             AS origin_name, 
            ontime.Origin                           AS origin_id, 
            dest.latitude_deg                       AS dest_lat, 
            dest.longitude_deg                      AS dest_lon, 
            dest.name                               AS dest_name,
            ontime.Dest                             AS dest_id,
            COUNT(*)                                AS number_of_flights,
            SUM(ontime.DepDelay > 15)               AS delay_over_15,
            AVG(ontime.DepDelay)                    AS DepDelay, 
            AVG(ontime.ArrDelay - ontime.DepDelay)  AS delay_gain 
    FROM ontime 
    LEFT JOIN airports AS origin ON ontime.Origin == origin.iata
    LEFT JOIN airports AS dest   ON ontime.Dest   == dest.iata
    WHERE origin.country == 'US' AND dest.country == 'US' 
    GROUP BY ontime.year,
            ontime.Origin,
            ontime.Dest
""")
conn.commit()

In [2]:
conn.execute("""
DROP TABLE US_filghts_dep_delay_Y 
""")
conn.commit()

In [4]:
cur = conn.execute("""
SELECT * FROM US_filghts_dep_delay_Y where year == 2000 AND dest_id == 'JFK'
""")
cur.fetchall()

[(2000,
  33.63669967651367,
  -84.4281005859375,
  'Hartsfield Jackson Atlanta International Airport',
  'ATL',
  40.63980103,
  -73.77890015,
  'John F Kennedy International Airport',
  'JFK',
  1205,
  238,
  12.774273858921163,
  0.13692946058091288),
 (2000,
  41.9388999939,
  -72.68319702149999,
  'Bradley International Airport',
  'BDL',
  40.63980103,
  -73.77890015,
  'John F Kennedy International Airport',
  'JFK',
  175,
  25,
  0.9485714285714286,
  12.742857142857142),
 (2000,
  42.36429977,
  -71.00520325,
  'General Edward Lawrence Logan International Airport',
  'BOS',
  40.63980103,
  -73.77890015,
  'John F Kennedy International Airport',
  'JFK',
  1997,
  566,
  13.111166750125188,
  1.3475212819228843),
 (2000,
  39.0488014221,
  -84.6678009033,
  'Cincinnati Northern Kentucky International Airport',
  'CVG',
  40.63980103,
  -73.77890015,
  'John F Kennedy International Airport',
  'JFK',
  830,
  151,
  12.398795180722892,
  2.4542168674698797),
 (2000,
  38.8521

In [10]:
cur = conn.execute("""
SELECT dest_id, dest_name FROM US_filghts_dep_delay_Y group by dest_id order by number_of_flights
""")
cur.fetchall()

[('AMA', 'Rick Husband Amarillo International Airport'),
 ('BFI', 'Boeing Field King County International Airport'),
 ('CBM', 'Columbus Air Force Base'),
 ('CHS', 'Charleston Air Force Base-International Airport'),
 ('GRB', 'Austin Straubel International Airport'),
 ('LBB', 'Lubbock Preston Smith International Airport'),
 ('MCI', 'Kansas City International Airport'),
 ('MGM', 'Montgomery Regional (Dannelly Field) Airport'),
 ('MSN', 'Dane County Regional Truax Field'),
 ('OKC', 'Will Rogers World Airport'),
 ('PIA', 'General Wayne A. Downing Peoria International Airport'),
 ('RFD', 'Chicago Rockford International Airport'),
 ('SAV', 'Savannah Hilton Head International Airport'),
 ('SKA', 'Fairchild Air Force Base'),
 ('SPI', 'Abraham Lincoln Capital Airport'),
 ('LGA', 'La Guardia Airport'),
 ('AUS', 'Austin Bergstrom International Airport'),
 ('DSM', 'Des Moines International Airport'),
 ('AGS', 'Augusta Regional At Bush Field'),
 ('BGR', 'Bangor International Airport'),
 ('SRQ', 'Sar

In [5]:
# tabele wynikowe
import pandas as pd
import sqlite3

conn_processed = sqlite3.connect("/home/pkosewski/python/pd4/databases/map.sql")
conn           = sqlite3.connect("/home/pkosewski/python/pd4/databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from airports", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("airports", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()

In [4]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("/home/pkosewski/python/pd4/databases/map.sql")


cur = conn.execute("""
SELECT count(*) FROM US_filghts_dep_delay_Y
""")
cur.fetchall()

[(64078,)]

# Daily patterns


In [1]:
import pandas as pd
import sqlite3

conn_processed = sqlite3.connect("/home/pkosewski/python/pd4/databases/processed.sql")
conn           = sqlite3.connect("/home/pkosewski/python/pd4/databases/flights_full.sql")


In [5]:
conn.execute("""
DROP TABLE Arr_Delays_vs_time_h
""")
conn.commit()

In [7]:
conn.execute("""
CREATE TABLE Arr_Delays_vs_time_h AS 
    SELECT  CASE 
               WHEN ontime.DepTime == 2400 
                  THEN 0
                  ELSE ontime.DepTime 
            END                                     AS time,
            ontime.UniqueCarrier                    AS UniqueCarrier, 
            COUNT(*)                                AS number_of_flights,
            AVG(ontime.DepDelay)                    AS DepDelay, 
            AVG(ontime.ArrDelay)                    AS ArrDelay 
    FROM ontime
    WHERE ontime.DepTime IS NOT NULL           AND
            typeof(ontime.DepTime) = "integer" AND
            ontime.DepTime % 100 < 60          AND
            ontime.DepTime <= 2400
    GROUP BY time,
            ontime.UniqueCarrier
    HAVING DepDelay IS NOT NULL                AND
            ArrDelay IS NOT NULL
    ORDER BY  UniqueCarrier, time
""")
conn.commit()

In [4]:
cur = conn.execute("""
SELECT count(*) FROM Arr_Delays_vs_time_h
""")
cur.fetchall()

[(38662,)]

In [8]:
cur = conn.execute("""
select count(*) From (
SELECT UniqueCarrier FROM Arr_Delays_vs_time_h group by UniqueCarrier
)
""")
cur.fetchall()

[(29,)]

In [21]:
cur = conn.execute("""
SELECT UniqueCarrier FROM Arr_Delays_vs_time_h group by UniqueCarrier
""")
x = cur.fetchall()
for y in x: 
    p = str(y)
    print(p)

('9E',)
('AA',)
('AQ',)
('AS',)
('B6',)
('CO',)
('DH',)
('DL',)
('EA',)
('EV',)
('F9',)
('FL',)
('HA',)
('HP',)
('ML (1)',)
('MQ',)
('NW',)
('OH',)
('OO',)
('PA (1)',)
('PI',)
('PS',)
('TW',)
('TZ',)
('UA',)
('US',)
('WN',)
('XE',)
('YV',)


In [5]:
conn.execute("""
DROP TABLE Arr_Delays_vs_time_h24102007
""")
conn.commit()

In [6]:
conn.execute("""
CREATE TABLE Arr_Delays_vs_time_h24102007 AS 
    SELECT  CASE 
               WHEN ontime.CRSDepTime == 2400 
                  THEN 0
                  ELSE ontime.CRSDepTime 
            END                                     AS time,
            ontime.UniqueCarrier                    AS UniqueCarrier, 
            COUNT(*)                                AS number_of_flights,
            AVG(ontime.ArrDelay)                    AS ArrDelay 
    FROM ontime
    WHERE ontime.CRSDepTime IS NOT NULL           AND
            typeof(ontime.CRSDepTime) = "integer" AND
            ontime.CRSDepTime % 100 < 60          AND
            ontime.CRSDepTime <= 2400             AND
            ontime.year == 2007                AND
            ontime.month == 10                 AND
            ontime.DayofMonth == 24
    GROUP BY time, ontime.UniqueCarrier
    HAVING ArrDelay IS NOT NULL
    ORDER BY  UniqueCarrier, time
""")
conn.commit()

In [3]:
cur = conn.execute("""
SELECT count(*) FROM Arr_Delays_vs_time_h24102007
""")
cur.fetchall()

[(11037,)]

In [1]:
# tabele wynikowe
import pandas as pd
import sqlite3

conn_processed = sqlite3.connect("/home/pkosewski/python/pd4/databases/hours.sql")
conn           = sqlite3.connect("/home/pkosewski/python/pd4/databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from Arr_Delays_vs_time_h24102007", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("Arr_Delays_vs_time_h24102007", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()

In [2]:
# tabele wynikowe
import pandas as pd
import sqlite3

conn_processed = sqlite3.connect("/home/pkosewski/python/pd4/databases/hours.sql")
conn           = sqlite3.connect("/home/pkosewski/python/pd4/databases/flights_full.sql")

# Load the data into a DataFrame
df = pd.read_sql_query("SELECT * from Arr_Delays_vs_time_h", conn)

# Write the new DataFrame to a new SQLite table
df.to_sql("Arr_Delays_vs_time_h", conn_processed, if_exists="replace")

conn.close()
conn_processed.close()